In [1]:
import cv2
import time
import glob
import torch
import string
import numpy as np

from os.path import join, sep
from torchvision import transforms
from torch.autograd import Variable

## Task-3 验证码预测

### 基于CNN的验证码识别

In [2]:
# !git clone https://github.com/braveryCHR/CNN_captcha.git ./model/cnn_captcha
# !rm -rf ./model/cnn_captcha/.git

### 基于CRNN的验证码识别

In [3]:
# !git clone https://github.com/tinggh/captcha_recognition.git ./model/crnn_captcha
# !rm -rf ./model/crnn_captcha/.git

In [4]:
from model.crnn_captcha import CRNN, utils

In [5]:
crnn_model_path = join('.', 'saved_models', 'crnn', 'crnn_captcha.pth')
test_img_path = join('.', 'test', 'recaptcha', 'pictures')  
# 得用仓库里的图片正确率才高不知道是为什么
alphabet = string.digits + string.ascii_letters
nclass = len(alphabet)+1

transformer = transforms.Normalize([0.906, 0.910, 0.907], [0.147, 0.130, 0.142])
imgH = 64

In [6]:
def crnn_recognition(image, model):

    converter = utils.strLabelConverter(alphabet)
    h, w, c = image.shape
    ratio = imgH * 1.0 / h
    image = cv2.resize(image, (0, 0), fx=ratio, fy=ratio, 
                       interpolation=cv2.INTER_CUBIC)
    image = (np.reshape(image, (imgH, -1, c))).transpose(2, 0, 1)
    image = image.astype(np.float32) / 255.
    image = torch.from_numpy(image).type(torch.FloatTensor)
    image = transformer(image)
    if torch.cuda.is_available():
        image = image.cuda()
    image = image.view(1, *image.size())
    image = Variable(image)

   
    preds = model(image)

    _, preds = preds.max(2)
    preds = preds.transpose(1, 0).contiguous().view(-1)

    preds_size = Variable(torch.IntTensor([preds.size(0)]))
    sim_pred = converter.decode(preds.data, preds_size.data, raw=False)
    return sim_pred

In [7]:
model = CRNN(imgH, 3, nclass, 256)
if torch.cuda.is_available():
    model = model.cuda()
print('loading pretrained model from {0}'.format(crnn_model_path))
model.load_state_dict(torch.load(crnn_model_path))
model.eval()
started = time.time()
img_paths = glob.glob(join(test_img_path, '*.png'))
n = 0
correct, total = 0, 0
for i, img_path in enumerate(img_paths):
    img_name = img_path.split(sep)[-1].rstrip('.png')
    image = cv2.imread(img_path)
    pred = crnn_recognition(image, model)
    print('path: {0}, pred: {1}'.format(img_name, pred))
    if pred == img_name:
        correct += 1
    total += 1
finished = time.time()
print('elapsed time: {0:.4f}s'.format(finished-started))
print('accuracy: {0:.6f}%'.format(correct / total * 100))

loading pretrained model from .\saved_models\crnn\crnn_captcha.pth
path: 003N, pred: osN
path: 009C, pred: oec
path: 00BN, pred: coaN
path: 00G4, pred: ooG4
path: 022R, pred: z
path: 026E, pred: ozc
path: 02FM, pred: o2FM
path: 040N, pred: o4oN
path: 05TJ, pred: osTs
path: 06AB, pred: eAE
path: 06HC, pred: o76Hc
path: 072A, pred: oz4
path: 07G0, pred: 7G
path: 07PV, pred: o7Pv
path: 086K, pred: ozeK
path: 08MX, pred: c
path: 09CB, pred: oqcB
path: 09S7, pred: o9s7
path: 0A4A, pred: 444
path: 0BYG, pred: osxc
path: 0DAI, pred: oDA
path: 0F07, pred: oF1
path: 0F2W, pred: oFW
path: 0FA7, pred: 0FAz
path: 0GDA, pred: oGA
path: 0GPU, pred: sGPu
path: 0GUK, pred: oGsKK
path: 0HF0, pred: oHF0
path: 0HH9, pred: H9
path: 0HT2, pred: oHxz
path: 0IDH, pred: Iof
path: 0J7F, pred: oJzc
path: 0JWT, pred: oJWT
path: 0K3R, pred: 0K3R
path: 0LI6, pred: Ic
path: 0LPW, pred: LPW
path: 0MQ9, pred: oM
path: 0N6D, pred: oN6Do
path: 0NS5, pred: rNss
path: 0P2F, pred: 0PzF
path: 0QVL, pred: cvL
path: 0RPM, pr